In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import gc
from IPython.display import clear_output

from functools import lru_cache

def sum_2_tuples(a, b):
    return tuple(map(sum, zip(a, b)))

File paths:

In [2]:
LOB_DIR = '../Data_lob/'

TOXIC_BUY_MOMENTS_PATH = '../0_CommonFiles/toxic/toxic_buy_moments_1m_02_06.feather'
TOXIC_SELL_MOMENTS_PATH = '../0_CommonFiles/toxic/toxic_sell_moments_1m_02_06.feather'
TWAP_FILE = './twap.db3'

Experiment details:

In [3]:
# TWAP hedger
volume = 5_000 # in contracts
share_1_hedge = 0.2
twap_interval_seconds = 5

# Read and prepare 'toxic' moments

In [4]:
toxic_buy_moments = pd.read_feather(TOXIC_BUY_MOMENTS_PATH)

In [5]:
toxic_buy_moments_2 = pd.DataFrame(index=toxic_buy_moments['Time']).between_time('10:00', '23:50')

In [6]:
toxic_buy_moments_2

""
Time
2021-02-01 10:01:36.771
2021-02-01 10:05:02.883
2021-02-01 10:05:11.061
2021-02-01 10:05:23.526
2021-02-01 10:07:27.929
...
2021-06-30 19:17:20.554
2021-06-30 19:30:16.189
2021-06-30 19:31:46.414


In [7]:
toxic_buy_moments_2_index = toxic_buy_moments_2.index

In [8]:
toxic_sell_moments = pd.read_feather(TOXIC_SELL_MOMENTS_PATH)

In [9]:
toxic_sell_moments_2 = pd.DataFrame(index=toxic_sell_moments['Time']).between_time('10:00', '23:50')

In [14]:
toxic_sell_moments_2

""
Time
2021-02-01 10:00:09.015
2021-02-01 10:04:36.332
2021-02-01 10:04:57.946
2021-02-01 10:05:19.015
2021-02-01 10:08:27.788
...
2021-06-30 19:03:41.640
2021-06-30 19:05:10.360
2021-06-30 19:05:24.713


In [10]:
toxic_sell_moments_2_index = toxic_sell_moments_2.index

# Functions

In [11]:
class LOBCache:
    def __init__(self):
        pass
    
    def get_lob(self, datetime):
        filename = datetime.strftime('LOB_%m%d.feather')
        return self.__get_lob_by_filename(filename)
        pass
    
    @lru_cache(maxsize=1)
    def __get_lob_by_filename(self, filename):
        print(filename)
        lob = pd.read_feather(LOB_DIR + filename).dropna()
        lob['BID_SIZE10'] = 99_999_999
        lob['ASK_SIZE10'] = 99_999_999
        return lob

In [12]:
def make_hedge(lob, datetime, size, side):
    #lob = lob[lob['Time'] >= datetime]
    index = lob.Time.searchsorted(datetime, side = 'right')
    lob = lob.iloc[index:]
    if len(lob) == 0:
        return None, lob
    row = lob.iloc[0]
    
    quote_side = 'BID' if (side == 'S') else 'ASK'
    
    maxsize = sum([row[quote_side + '_SIZE' + str(level)] for level in range(1, 11)])
        
    if maxsize < size:
        raise ValueError(f'size {size} is greater than available {maxsize}')
    
    size_hedged, sum_hedged = 0, 0.0
    level = 1
    while size_hedged < size:
        portion = min(size - size_hedged, row[quote_side + '_SIZE' + str(level)])
        price = row[quote_side + '_PRICE' + str(level)]
        size_hedged += portion
        sum_hedged += portion * price
        print(f'added {portion} by {price}')
        level += 1
        
    print('HEDGE ended')
    return (size_hedged, sum_hedged), lob

In [13]:
cache = LOBCache()

In [14]:
def calc_hedge_twap(cache, moment, interval_sec):
    lob = cache.get_lob(moment)
    reduced_lob = lob
    remain = volume
    total = None
    
    if len(reduced_lob) == 0:
        return None
    
    for seconds in range(0, 100, interval_sec):
        time_param = moment + np.timedelta64(seconds, 's')
        print(time_param)

        size_contracts = min(int(volume * share_1_hedge), remain)
        hedge_result, reduced_lob = make_hedge(reduced_lob, time_param, size = size_contracts, side = side)
        while hedge_result is None and len(reduced_lob) > 0:
            print('bad orderbook', reduced_lob.iloc[0])
            reduced_lob = reduced_lob.iloc[1: , :]
            hedge_result, reduced_lob = make_hedge(reduced_lob, time_param, size = size_contracts, side = side)

        if hedge_result is None:
            break
            
        if total is None:
            total = hedge_result
        else:
            #print('total='+str(total))
            #print('hedge_result='+str(hedge_result))
            total = sum_2_tuples(total, hedge_result)
        remain -= hedge_result[0]
        if remain == 0:
            break
            
    #del lob
            
    if remain > 0:
        return None
    else:
        return total

# Calculation

In [15]:
#CREATE TABLE RESULT(MOMENT TEXT, SIDE TEXT, QUANTITY INTEGER, SUM REAL, PRIMARY KEY(MOMENT,SIDE));
twap_sql = sqlite3.connect(TWAP_FILE)
twap_sql.execute('CREATE TABLE IF NOT EXISTS RESULT(MOMENT TEXT, SIDE TEXT, QUANTITY INTEGER, SUM REAL, PRIMARY KEY(MOMENT,SIDE));')
cur=twap_sql.cursor()

In [16]:
records = []

for side in ('B', 'S'):
    if side == 'B':
        moments = toxic_buy_moments_2_index
    elif side == 'S':
        moments = toxic_sell_moments_2_index
        
    for moment in moments:
        record = (str(moment),side)
        query='select exists(select 1 from result where moment=? and side=? collate nocase) limit 1'
        # 'query' RETURNS 1 IF USERNAME EXISTS OR 0 IF NOT, AS INTEGER(MAYBE). 'collate nocase'= CASE INSENSITIVE, IT'S OPTIONAL
        check=cur.execute(query,record) 
        if check.fetchone()[0]==1:
            print(f'Moment already available ' + str(record))
            continue
        result = calc_hedge_twap(cache, moment, interval_sec=twap_interval_seconds)

        query='INSERT OR REPLACE INTO result(moment, side, quantity, sum) VALUES(?, ?, ?, ?);'
        if result:
            record = (str(moment), side, int(result[0]), result[1])
        else:
            record = (str(moment), side, -1, -1)

        print(record)
        cur.execute(query, record)
        records.append(record)

        gc.collect()

        if len(records) % 100 == 0:
            clear_output(wait=False)

2021-06-30 15:00:59.309000
added 553 by 72.93
added 447 by 72.9275
HEDGE ended
2021-06-30 15:01:04.309000
added 94 by 72.9325
added 903 by 72.93
added 3 by 72.9275
HEDGE ended
2021-06-30 15:01:09.309000
added 738 by 72.93
added 262 by 72.9275
HEDGE ended
2021-06-30 15:01:14.309000
added 150 by 72.9325
added 850 by 72.93
HEDGE ended
2021-06-30 15:01:19.309000
added 44 by 72.9325
added 741 by 72.93
added 215 by 72.9275
HEDGE ended
('2021-06-30 15:00:59.309000', 'S', 5000, 364648.4025)
2021-06-30 15:15:36.235000
added 100 by 72.9925
added 312 by 72.99
added 300 by 72.9875
added 288 by 72.985
HEDGE ended
2021-06-30 15:15:41.235000
added 1 by 72.9825
added 221 by 72.98
added 300 by 72.9775
added 320 by 72.975
added 158 by 72.9725
HEDGE ended
2021-06-30 15:15:46.235000
added 10 by 72.965
added 200 by 72.9625
added 237 by 72.96
added 350 by 72.9575
added 203 by 72.955
HEDGE ended
2021-06-30 15:15:51.235000
added 9 by 72.965
added 700 by 72.9625
added 237 by 72.96
added 54 by 72.9575
HEDGE end

added 200 by 72.9075
added 501 by 72.905
added 299 by 72.9025
HEDGE ended
2021-06-30 16:09:54.515000
added 200 by 72.9125
added 450 by 72.91
added 350 by 72.9075
HEDGE ended
2021-06-30 16:09:59.515000
added 350 by 72.91
added 650 by 72.9075
HEDGE ended
2021-06-30 16:10:04.515000
added 497 by 72.91
added 503 by 72.9075
HEDGE ended
('2021-06-30 16:09:44.515000', 'S', 5000, 364536.6225)
2021-06-30 16:32:03.575000
added 150 by 72.905
added 460 by 72.9025
added 390 by 72.9
HEDGE ended
2021-06-30 16:32:08.575000
added 150 by 72.905
added 460 by 72.9025
added 390 by 72.9
HEDGE ended
2021-06-30 16:32:13.575000
added 150 by 72.905
added 360 by 72.9025
added 490 by 72.9
HEDGE ended
2021-06-30 16:32:18.575000
added 150 by 72.9075
added 400 by 72.905
added 450 by 72.9025
HEDGE ended
2021-06-30 16:32:23.575000
added 66 by 72.91
added 200 by 72.9075
added 350 by 72.905
added 384 by 72.9025
HEDGE ended
('2021-06-30 16:32:03.575000', 'S', 5000, 364514.57)
2021-06-30 16:47:08.656000
added 344 by 72.962

2021-06-30 18:01:55.215000
added 100 by 73.04
added 700 by 73.0375
added 200 by 73.035
HEDGE ended
2021-06-30 18:02:00.215000
added 200 by 73.0425
added 250 by 73.04
added 400 by 73.0375
added 150 by 73.035
HEDGE ended
2021-06-30 18:02:05.215000
added 100 by 73.0225
added 250 by 73.02
added 350 by 73.0175
added 300 by 73.015
HEDGE ended
2021-06-30 18:02:10.215000
added 298 by 73.0275
added 100 by 73.025
added 250 by 73.0225
added 150 by 73.02
added 150 by 73.0175
added 52 by 73.015
HEDGE ended
2021-06-30 18:02:15.215000
added 300 by 73.04
added 300 by 73.0375
added 400 by 73.035
HEDGE ended
('2021-06-30 18:01:55.215000', 'S', 5000, 365153.85)
2021-06-30 18:02:01.964000
added 347 by 73.04
added 500 by 73.0375
added 153 by 73.035
HEDGE ended
2021-06-30 18:02:06.964000
added 100 by 73.0225
added 150 by 73.02
added 250 by 73.0175
added 100 by 73.015
added 200 by 73.0125
added 200 by 73.01
HEDGE ended
2021-06-30 18:02:11.964000
added 446 by 73.0275
added 150 by 73.0225
added 150 by 73.02
ad

2021-06-30 19:40:36.010000
added 222 by 73.1375
added 275 by 73.135
added 230 by 73.1325
added 273 by 73.13
HEDGE ended
2021-06-30 19:40:41.010000
added 175 by 73.135
added 330 by 73.1325
added 327 by 73.13
added 168 by 73.1275
HEDGE ended
2021-06-30 19:40:46.010000
added 42 by 73.135
added 230 by 73.1325
added 427 by 73.13
added 250 by 73.1275
added 51 by 73.125
HEDGE ended
2021-06-30 19:40:51.010000
added 42 by 73.135
added 230 by 73.1325
added 327 by 73.13
added 350 by 73.1275
added 51 by 73.125
HEDGE ended
2021-06-30 19:40:56.010000
added 227 by 73.13
added 250 by 73.1275
added 389 by 73.125
added 134 by 73.1225
HEDGE ended
('2021-06-30 19:40:36.010000', 'S', 5000, 365650.87999999995)


In [17]:
twap_sql.commit()

In [18]:
cur.close()
twap_sql.close()